In [1]:
import sys
import os

# Ensure Python finds config.py
sys.path.append(os.path.abspath(".."))  # Moves up one level to find config.py

In [2]:
import os
import json
import requests
import pandas as pd
import importlib
from config import RAW_DATA_DIR  # Import directory structure

# Load API key from local JSON file
api_key_path = os.path.expanduser("~/Documents/research/API/api_keys.json")
with open(api_key_path, "r") as f:
    fred_api_key = json.load(f)["FRED_API_KEY"]

def fetch_fred_data(series_id, api_key, start=None, end=None):
    """Fetches a single FRED series in JSON format and returns a DataFrame."""
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json"
    if start and end:
        url += f"&observation_start={start}&observation_end={end}"

    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    if "observations" not in data:
        return None

    df = pd.DataFrame(data["observations"])
    df["date"] = pd.to_datetime(df["date"])
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df.drop(columns=["realtime_start", "realtime_end"], errors="ignore", inplace=True)
    df.set_index("date", inplace=True)

    return df.rename(columns={"value": series_id})

def download_fred_data(api_key, start="2000-01-01", end="2025-01-01"):
    """Fetches key FRED macroeconomic indicators and saves the raw dataset as a CSV."""
    series_dict = {
        "CPIAUCSL":  "CPI",
        "UNRATE":    "UNRATE",
        "FEDFUNDS":  "FEDFUNDS",
        "INDPRO":    "INDPRO",
        "M2SL":      "M2",
        "GDPC1":     "REAL_GDP",
        "GDPPOT":    "POT_GDP",
    }

    all_data = {colname: fetch_fred_data(sid, api_key, start, end) for sid, colname in series_dict.items()}
    all_data = {k: v for k, v in all_data.items() if v is not None}

    if not all_data:
        return

    df_raw = pd.concat(all_data.values(), axis=1, join="outer")
    
    # Ensure the raw data directory exists
    os.makedirs(RAW_DATA_DIR, exist_ok=True)
    
    out_file = os.path.join(RAW_DATA_DIR, "macro_data_raw.csv")
    df_raw.to_csv(out_file)

if __name__ == "__main__":
    download_fred_data(api_key=fred_api_key, start="2004-01-01", end="2025-01-01")
